# Before We Begin

You will need:
  1. a virtualenv
  2. Jupyter notebook running a kernel in that virtualenv
  3. Klein and Treq (and therefore Twisted) installed in that kernel

To get those:

```console
$ virtualenv abc
$ . abc/bin/activate
$ pip install jupyter[notebook] klein treq
$ jupyter notebook Twisted-Tutorial.ipynb
```

The first thing we need to do is to set up the klein application. If what follows seems a bit arcane, there is no need to worry: it is only needed to integrate Klein properly with Jupyter. Every way of deploying a Klein application will need, eventually, some sort of "rubber meets road" part, where we connect the application to the network. There will always be some complicating consideration: which low-level event loop to use? What kind of sockets to listen on? Here is the specific way we do it for this demo on Jupyter.

Another compromise done here is that the entire initialization is in one cell. From a pedagogical perspective, it would have been better to split it. However, this way it is much easier to hit "restart kernel" and then re-run the initialization. 

So what does happen here? (It is safe to skip this part) Jupyter is running on top of the Tornado event loop. Twisted can run its "reactor", the Twisted event loop, on top of Tornado's. This is not the default -- the default is the native-Twisted event loop. The first thing we do is install the Tornado reactor. It is important this is the first thing to be done: Twisted will automatically install the default reactor as soon as a reactor is requested, if one is not already installed.

Then, we created a server endpoint which listens on port 8080. We get the global Klein resource, wrap it in a Site object and connect it to the endpoint. Note that for production use of Klein, it is best not to use the global resource. However, for a quick prototype, it is extremely useful.

In [1]:
from tornado.platform.twisted import install
reactor = install()

from twisted.web.server import Site
from twisted.internet import endpoints
from klein import route, resource

description = "tcp:8080"

ep = endpoints.serverFromString(reactor, description)
s = Site(resource())
ep.listen(Site(resource()))

<Deferred at 0x10b547518 current result: <<class 'twisted.internet.tcp.Port'> of twisted.web.server.Site on 8080>>

As is traditional, the first thing to be done in any programming environment is to greet. This allows both showing what is the "minimum program" as well as have an easy way to demonstrate the environment is working. Klein uses decorators to indicate routing. We put a resource at the root which greets with a constant string.

In [3]:
from klein import route
@route('/')
def home(request):
    return 'Hello, everybody!'

The excellent webbrowser module can be used to open a browser to a given page. If this is running locally (and not through a virtualization environment or a cloud environment), the following will open a browser to a pleasant (if generic) greeting from Klein.

In [4]:
import webbrowser
webbrowser.open("http://127.0.0.1:8080/")

True

Constant strings, though, are boring. The reason to use a web application framework is, well, to write applications. We personalize the greeting by using string interpolation. Maybe it is not the fanciest templating solution, but it will do for now.

In [5]:
@route('/greet/<user>')
def greet(request, user):
    return 'Hello {}!'.format(user.upper())

Try changing the URL below to have your name (or your nickname, or that of a friend that needs to be shown the power of Klein and Twisted).

In [6]:
webbrowser.open("http://localhost:8080/greet/your-name")

True

Up until now, the code would have been similar if a WSGI framework like 

In [3]:
import json
from twisted.internet import defer
import treq

@route('/ip')
@defer.inlineCallbacks
def getip(request):
    url = 'https://httpbin.org/ip'
    response = yield treq.get(url)
    content = yield response.content()
    value = json.loads(content)
    ip = value['origin']
    defer.returnValue('From {}'.format(ip))

In [6]:
import webbrowser
webbrowser.open("http://127.0.0.1:8080/ip")

True

In [7]:
@route('/compare')
@defer.inlineCallbacks
def compare(request):
    url = 'https://httpbin.org/delay/10'
    response = yield treq.get(url)
    content = yield response.content()
    value = json.loads(content)
    first_ip = value['origin']
    response = yield treq.get(url)
    content = yield response.content()
    value = json.loads(content)
    second_ip = value['origin']    
    defer.returnValue('These are the same:{}=={}'.format(first_ip, second_ip))

In [8]:
webbrowser.open("http://127.0.0.1:8080/compare")

True

In [8]:
@route('/faster-compare')
def faster_compare(request):
    url = 'https://httpbin.org/delay/10'
    d = treq.get(url)
    d.addCallback(lambda response: response.content())
    d.addCallback(json.loads)
    d.addCallback(lambda dct: dct['origin'])
    d2 = treq.get(url)
    d2.addCallback(lambda response: response.content())
    d2.addCallback(json.loads)
    d2.addCallback(lambda dct: dct['origin'])
    both = defer.gatherResults([d, d2])
    def do_format(ips):
        first_ip, second_ip = ips
        return 'These are the same:{}=={}'.format(first_ip, second_ip)
    both.addCallback(do_format)
    return both

In [9]:
webbrowser.open("http://127.0.0.1:8080/faster-compare")

True

In [10]:
from twisted.internet import task
import operator

@route('/add/<int:a>/<int:b>')
def add(request, a, b):
    d = task.deferLater(reactor, 2, operator.add, a, b)
    d.addCallback(str)
    return d

In [11]:
webbrowser.open("http://127.0.0.1:8080/add/3/4")

True

In [16]:
def remote_add(x, y):
    url = 'http://localhost:8080/add'
    d = treq.get(url + '/{}/{}'.format(x, y))
    d.addCallback(lambda response: response.content())
    return d

@route('/add4/<int:a>/<int:b>/<int:c>/<int:d>')
def add4(request, a, b, c, d):
    d = defer.gatherResults([remote_add(a, b), 
                             remote_add(c, d)])
    @d.addCallback
    def final_add(results):
        x, y = results
        return remote_add(x, y)
    return d

@route('/slow-add4/<int:a>/<int:b>/<int:c>/<int:d>')
@defer.inlineCallbacks
def slow_add4(request, a, b, c, d):
    x = yield remote_add(a, b)
    y = yield remote_add(c, d)
    res = yield remote_add(x, y)
    defer.returnValue(str(res))

In [17]:
webbrowser.open("http://127.0.0.1:8080/add4/3/4/9/14")
webbrowser.open("http://127.0.0.1:8080/slow-add4/3/4/9/14")

True

In [18]:
counter = 0
@route('/count')
def count(request):
    global counter
    counter += 1
    return str(counter)

In [19]:
webbrowser.open("http://127.0.0.1:8080/count")
webbrowser.open("http://127.0.0.1:8080/count")
webbrowser.open("http://127.0.0.1:8080/count")

True

In [20]:
from twisted.internet import task
loops = 0
def add_loop():
    global loops
    loops += 1

@route('/loops')
def show_loops(request):
    return str(loops)

lc = task.LoopingCall(add_loop)
d = lc.start(5)

In [21]:
reactor.callLater(6, webbrowser.open, "http://127.0.0.1:8080/loops")
reactor.callLater(12, webbrowser.open, "http://127.0.0.1:8080/loops")

In [25]:
blocking = []

@route('/block')
def count(request):
    result = defer.Deferred()
    blocking.append(result)
    result.addCallback(lambda x: x.upper())
    return result

@route('/unblock/<string:value>')
def unblock(request, value):
    blocking.pop(0).callback(value)
    return u"OK " + value

In [28]:
webbrowser.open("http://127.0.0.1:8080/block")

True

In [29]:
webbrowser.open("http://127.0.0.1:8080/unblock/hello%20universe")

True